# Процессы 

Итак, каждый процесс, который запущен в операционной системе, имеет свои характеристики. Одна из главных характеристик — это идентификатор процесса, или PID. При помощи PID можно узнать многое о процессе. Каждый процесс занимает некий объем оперативной памяти. Он запрашивает её у системы, она ему её возвращает и аллоцирует. Также у процесса есть стек. Стек используется для вызова функций, для создания локальных переменных у этих функций. А также у каждого процесса есть список открытых файлов, стандартный ввод и стандартный вывод. В своих примерах я буду использовать операционную систему класса Linux и Python 3. Давайте попробуем узнать, какие процессы запущены в нашей операционной системе. Для этого нам потребуется консоль и команда top. Набираем команду top, вводим enter и видим результаты в виде таблицы. Команда top отображает нам список процессов, которые сейчас функционируют в операционной системе

In [ ]:
# process_ex.py

import time
import os

pid = os.getpid()

while True:
    print(pid, time.time())
    time.sleep(2)



В терминале:
- top - список процессов
- ps aux - список всех процессов
- ps aux | grep process_ex.py - отфильтровать наш процесс
- ps aux | head -1; ps aux | grep process_ex.py - вывести шапку
- strace -p 6575 - системные вызовы процесса
- lsof -p 6575 - список файло используемых процессом
- python3 process_ex.py > data/log.txt - перенаправляем поток ввода вывода в файл

Давайте поймем, какую же последовательность команд выполняет наш процесс. Что он вообще делает? Вообще, процессы, когда выполняются в операционной системе, они делают системные вызовы. Системные вызовы выполняют непосредственно ядро операционной системы, а результаты этих системных вызовов возвращаются к процессу, который их вызвал. Например, вывод в консоль, или стандартный вывод — это системный вызов. Для того чтобы посмотреть, какие системные вызовы делает наш процесс, можно воспользоваться командой strace, указать ей pid нашего процесса. Нужны дополнительные права для этой команды

Давайте посмотрим еще на список файлов, которые открыты в нашем процессе. Для этого мы можем воспользоваться командой lsof

Опять же, указываем pid нашего процесса. Список достаточно большой. Как минимум, когда наш Python интерпретатор запускает нашу программу, открываются дополнительные системные библиотеки. Они отображены тоже в результатах команды lsof. Но самое главное, что нас сейчас интересует, — это стандартный поток ввода, вывода и поток ошибок. Это файловые дескрипторы 0, 1 и 2. Мы видим, что они равны терминалу. Давайте попробуем прервать выполнение программы при помощи Ctrl+C и перенаправим стандартный вывод в файл. Делается это при помощи такой команды в любой файл. Снова поищем наш процесс при помощи команды ps. Его pid = 2367. И при помощи lsof выведем список открытых файлов. Результат команды lsof доказывает нам, что стандартный вывод у нашего процесса поменялся на файл

### Создание процессов


In [ ]:
# process_ex1.py

import time
import os

pid = os.fork()
if pid == 0:
    # дочерний процесс
    while True:
        print('child:', os.getpid())
        time.sleep(5)
else:
    # родительский процесс
    print('parent:', os.getpid())
    os.wait()

Процесс в операционной системе создается при помощи системного вызова fork. Давайте рассмотрим программу, которая создает дочерний процесс при помощи системного вызова fork. Я ее привел на слайде. Импортируем пару модулей time и os, затем вызываем системный вызов fork. Системный вызов fork создает точную копию родительского процесса. Это означает, что вся память, все файловые дискрипторы и все ресурсы, которые были доступны в родительском процессе, будут целиком и полностью скопированы в дочернем процессе. То есть после того, как системный вызов fork отработал, с этого момента у нас два процесса в операционной системе. Единственное отличие заключается в том, что системный вызов fork в родительский процесс вернет pid дочернего процесса, а в дочернем процессе переменная pid будет равна нулю. Код, который я сейчас выделил, будет исполнен в дочернем процессе, а код, который находится за веткой else, будет исполнен в родительском процессе. Итак, в родительском процессе мы вызываем системный вызов os.wait, это еще один дополнительный системный вызов и он позволяет нам дожидаться завершения дочернего процесса созданного. А в дочернем процессе вна бесконечном цикле выводим pid нашего процесса, который создали, и спим пять секунд